 We are going to use the langraph to work on this multi ai agent project which this langraph is same like graph database principle which are nodes ,states and nodes are connected to each other so same like that when we are working on complex architecture like creating multi Ai agent  it should be build by langraph for this project iam considering the nodes like 1) Start 2) Query router (returns correct datasource based on user query) 3) LLM 4) End and these 4 nodes we are going to connect to create this complex project 

In [ ]:
!pip install langchain langgraph cassio # cassio helps me to initialize the Astra Database

In [ ]:
import cassio

## initailizing the Connection of AStraDB
ASTRA_DB_APPLICATION_TOKEN="AstraCS:LTahzqdyQntsoamhHDFMEnfk:ad613b63aec2" # enter the "AstraCS:..." string found in in your Token JSON file"
ASTRA_DB_ID="c747b0-9594d14bacdf" # this id we can get it from AstraDB in Database section 
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [ ]:
!pip install langchain_community
!pip install -U tiktoken langchain-groq langchainhub langchain langgraph langchain_huggingface

In [ ]:
### Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader


### from langchain_cohere import CohereEmbeddings



# Docs to index
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
print(docs_list)

# Split The tiktoken encoder is a tokenizer and encoding tool
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)



In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
# Connecting the embeddings to Astradb database 
from langchain.vectorstores.cassandra import Cassandra
astra_vector_store=Cassandra(
    embedding=embeddings,
    table_name="qa_mini_demo", # In this table in Astradb iam going to store the vectors of documents 
    session=None,
    keyspace=None

)

In [ ]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper  # 
astra_vector_store.add_documents(doc_splits) # Whatever the documents we extracted from url we are going to save in astradb vector database by this line of code
print("Inserted %i headlines." % len(doc_splits))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store) # On top of astravector database i have created an index wrapper so that we can able to interact with vector database, in order to use vector database on top of index wrapper we need to use as_retriever()

In [ ]:
retriever=astra_vector_store.as_retriever()
retriever.invoke("what is agent")

In [ ]:
### Router is a path,  based on user query router tells in which way router path needs to go or recognise and responde appropriate (Vector knowledge database or wiki_search) w.rt user query

from typing import Literal  # It allows restricting a variable’s possible values to a specific set of literal values. In this case, it limits datasource to two valid options: "vectorstore" and "wiki_search".

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field  # RouteQuery Class A Pydantic data model (BaseModel) defining the expected structure of the output from the language model (LLM). Output must contain a datasource field with either "vectorstore" or "wiki_search".

# Data model
class RouteQuery(BaseModel): # RouteQuery A schema or data model that defines the structure of the output the LLM should produce
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "wiki_search"] = Field(
        ...,
        description="Given a user question choose to route it to wikipedia or a vectorstore.",
    )  # this datasource is key and Literal are the structred values  of output which llm needs to return any one of them based on user query + system context llm understand the whole context and it returns output in structured manner by adhering to Literals values as output


In [ ]:
# LLM with function call
from langchain_groq import ChatGroq
import os

groq_api_key="gsk_lMbTWeLY7HFG4mjQly5YRKT"
os.environ["GROQ_API_KEY"]=groq_api_key
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")

In [ ]:
# At the end of the day whatever output that I'm actually getting it should be in this particular format so here I'm going to create a variable called as structured lmore router is equal to llm dot with structured so there inside the llm is something called as with structured output and I'm going to use my route query over here okay I'm basically going  to call my route query class okay
structured_llm_router = llm.with_structured_output(RouteQuery)  # So in this way of structure here integrated the llm to the RouterQuery class, This is an LLM-based router, typically used to classify or decide on the next action based on the query. It uses a structured approach to output a decision or classification. For example: Determining the source of information: wiki_search vs. vectorstore. Routing questions based on domain expertise (e.g., technical vs. general queries). Outputting a structured response, such as: json Copy code { "datasource": "wiki_search", "confidence": 0.92 }

In [ ]:
# Prompt
system = """You are an expert at routing a user question to a vectorstore or wikipedia.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use wiki-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)


question_router = route_prompt | structured_llm_router  # Pipe (|) Operator The | operator creates a pipeline where the output of the first function (route_prompt) is passed as input to the second function (structured_llm_router).
 #------------- or ---------
#question_router = structured_llm_router(route_prompt(question))


In [ ]:
### Lets test the RouterQuery class 
print(
    question_router.invoke(
        {"question": "who is ms dhoni"}
    )
)

In [ ]:
print(question_router.invoke({"question": "What are the types of agent memory?"}))

By the above  2 user query my class RouterQuery is working well to choose the appropriate path or source(vector DB or wikipedia)  w.r.t user query 

In [ ]:
!pip install wikipedia

In [ ]:
### Working With Tools
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=1000)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
wiki.run("who is virat kohli")

In [ ]:
# I'm going to create a graph state which will be my class which is inheriting type dict and this will be in the form of key value pairs like question is equal to Sting generation is equal to  and documents list of s Str okay so I'm basically creating three variables this variables will be responsible in managing the entire state of AI AGent that I have actually created so it represents the state of a graph attributes are this many things okay so for right now just keep a note of that okay

## Graph

from typing import List

from typing_extensions import TypedDict # TypedDict is a Python construct that allows you to define the structure of a dictionary, including the expected keys and their data types.

 
class GraphState(TypedDict):  # The GraphState is a data structure that represents the state of the query processing workflow. It is used to store and manage information that flows through the nodes of the workflow. GraphState acts as a "state container" to track: What the user asked. The AI model's response. Any relevant documents retrieved during the process.
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str         # The user's query as a string (e.g., "What is agent?").
    generation: str       # The response or output generated by the LLM as a string.
    documents: List[str]  #  A list of documents (as strings) retrieved from the vector store or other sources like Wikipedia.

In [ ]:
##  Nodes
from langchain.schema import Document


def retrieve(state): # The retrieve function updates the GraphState by fetching relevant documents from the vector store for the given question.  the state we passed as parameter its a type of graph state which we defined in the above cell which the graph state have 3 variables which are question, generation, documents
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

In [ ]:
## Nodes
def wiki_search(state):
    """
    wiki search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---wikipedia---")
    print("---HELLO--")
    question = state["question"]
    print(question)

    # Wiki search
    docs = wiki.invoke({"query": question})
    #print(docs["summary"])
    wiki_results = docs
    wiki_results = Document(page_content=wiki_results)

    return {"documents": wiki_results, "question": question}

In [ ]:
### Edges ###


def route_question(state):
    """
    Route question to wiki search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")  # Key Points: Inputs: state: Contains the question. Process: Uses the question_router (integrated with an LLM) to decide where to send the query: wiki_search: For general knowledge or out-of-domain queries. vectorstore: For specific topics stored in the vector database. Output: Returns the next node to execute: "wiki_search" or "vectorstore".
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "wiki_search":
        print("---ROUTE QUESTION TO Wiki SEARCH---")
        return "wiki_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)
# Define the nodes
workflow.add_node("wiki_search", wiki_search)  # Key is the name of the node and wiki_search is the fucntion name of the node which contains the functionality of wiki_search which we defined in the above cell web search
workflow.add_node("retrieve", retrieve)  # retrieve  , same applies here

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,  
    {
        "wiki_search": "wiki_search",  # same like below
        "vectorstore": "retrieve",   #  here we define the condition in graph flow if the user query is related to vectorstore then corresponded value which is node name will get implemented
    },
) 

# after the user query corresponded flow got executed then now we need Edges to add the both node  flow edges to End Stage 
workflow.add_edge( "retrieve", END)  
workflow.add_edge( "wiki_search", END)
# Compile
app = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
from pprint import pprint

# Run
inputs = {
    "question": "What is an agent?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value['documents'][0].dict()['metadata']['description'])

In [ ]:
from pprint import pprint

# Run
inputs = {
    "question": "who is virat kohli"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value['documents'])

Your code snippet demonstrates how to integrate LangChain with AstraDB and build a sophisticated routing and querying system. Here's a summary and breakdown of its components:

---

### 1. **AstraDB Initialization**
You initialized a connection to an **AstraDB vector database** using the `cassio` library. This is critical for storing and retrieving embeddings.

### 2. **Document Loading and Preprocessing**
- Loaded documents from specified URLs using the `WebBaseLoader`.
- Split the documents into smaller chunks using the `RecursiveCharacterTextSplitter` with a `tiktoken` encoder for effective storage and retrieval in the vector database.

---

### 3. **Embeddings and Vector Database**
- Used **HuggingFaceEmbeddings** (`all-MiniLM-L6-v2`) to convert text chunks into vectors.
- Stored these vectors in the AstraDB vector store (`qa_mini_demo` table).

---

### 4. **Routing Logic**
- **RouterQuery Class**: A Pydantic model that determines whether a user query should go to the vector database or Wikipedia based on the content of the query.
- **LLM Integration**:
  - You integrated a **Groq-based LLM** to route questions intelligently.
  - Created a structured prompt to guide the model in deciding whether to route to the vector store or Wikipedia.

---

### 5. **Graph Workflow**
You built a **graph workflow** using LangGraph that manages the state and processes queries through nodes:
- **Nodes**:
  - `wiki_search`: Fetches information from Wikipedia.
  - `retrieve`: Retrieves relevant documents from the vector database.
- **Routing**:
  - Dynamically routes the user query based on the `question_router` output.
  - Routes to either `wiki_search` or `retrieve` depending on the question's context.
- **Graph Compilation**:
  - Defined `START` and `END` nodes for the workflow.
  - Dynamically decides the next node to call based on the routing logic.

---

### 6. **Running the Workflow**
- Input: A question such as *"What is agent?"*.
- Process:
  - The question is routed through the graph workflow.
  - Either fetches a document from the vector database or a Wikipedia summary.
- Output: Retrieved information based on the selected path.

---

### 7. **Additional Features**
- The `GraphState` class keeps track of:
  - The query (`question`).
  - LLM's generated responses (`generation`).
  - Retrieved documents (`documents`).

- **Final Output**:
  - The retrieved documents and their metadata (e.g., description) are displayed.



In [ ]:
{
  "clientId": "LTahDFMEnfk",
  "secret": "3NF_cE3vhSpBqH,lt",
  "token": "AstraCS:LTc561f5c3e9613b63aec2"
}

## Working explaiantion

### **Code Flow Explanation**

This code implements a **stateful workflow** for processing user queries using a graph-based model. The workflow routes the query to the appropriate node (either `wiki_search` or `retrieve`), processes the query, and collects relevant documents before ending the execution. Here's a detailed breakdown:

---

### **1. Key Components**

#### **a. `GraphState`**
- Defines the state structure of the workflow.
- It contains:
  - **`question`**: User's query (string).
  - **`generation`**: Output generated by the AI (string; initially empty).
  - **`documents`**: List of documents retrieved during the workflow (list of strings).

---

#### **b. Functions (Nodes)**
Each function corresponds to a "node" in the workflow graph.

1. **`route_question(state)`**
   - Determines the next step based on the query.
   - Routes queries to either:
     - `wiki_search`: For general knowledge.
     - `vectorstore`: For domain-specific information in the vector store.

2. **`wiki_search(state)`**
   - Fetches relevant results from Wikipedia for the given question.
   - Updates the `documents` field in the `GraphState`.

3. **`retrieve(state)`**
   - Retrieves documents related to the query from the vector store.
   - Updates the `documents` field in the `GraphState`.

---

#### **c. `StateGraph`**
- Implements the entire workflow as a directed graph with nodes and edges.
- Nodes correspond to the functions (`wiki_search`, `retrieve`).
- Edges represent transitions based on conditions (`route_question`).

---

### **2. Workflow Construction**

1. **Define Nodes**:
   - `wiki_search` for Wikipedia queries.
   - `retrieve` for vector store queries.

2. **Add Conditional Edges**:
   - The **START node** determines the route using `route_question`.
   - Routes:
     - `"wiki_search"` → `wiki_search` node.
     - `"vectorstore"` → `retrieve` node.

3. **Add Final Edges**:
   - Both `wiki_search` and `retrieve` lead to the **END node**.

4. **Compile Workflow**:
   - `app = workflow.compile()` creates an executable version of the workflow.

---

### **3. Code Execution**

#### **Input**
```python
inputs = {"question": "What is an agent?"}
```
- The user's query is "What is an agent?".

#### **Flow**

1. **Initial State**:
   - `GraphState` is initialized:
     ```python
     state = {
         "question": "What is an agent?",
         "generation": "",
         "documents": []
     }
     ```

2. **Routing Decision** (`route_question`):
   - The query is passed to `question_router`.
   - Assuming the query is domain-specific, the router returns `"vectorstore"`.

3. **Retrieve Documents** (`retrieve`):
   - The `retriever` fetches documents related to "What is an agent?".
   - Example:
     ```python
     state["documents"] = [
         "An agent is a software program that acts on behalf of a user.",
         "Agents in AI often refer to entities that perceive the environment and act."
     ]
     ```

4. **Final State**:
   - After processing, the state contains the updated `documents`:
     ```python
     {
         "question": "What is an agent?",
         "generation": "",
         "documents": [
             "An agent is a software program that acts on behalf of a user.",
             "Agents in AI often refer to entities that perceive the environment and act."
         ]
     }
     ```

---

### **4. Output Example**

During execution, each node’s state is printed:

#### **Step-by-Step Console Output**
```plaintext
Node 'route_question':
---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---

Node 'retrieve':
---RETRIEVE---
```

- **Final Output**:
   ```plaintext
   Node 'retrieve':
   Documents Retrieved:
   - "An agent is a software program that acts on behalf of a user."
   - "Agents in AI often refer to entities that perceive the environment and act."
   ```

---

### **5. Modifications for `wiki_search`**

If the query is routed to `wiki_search` (e.g., "Who is MS Dhoni?"), the workflow fetches Wikipedia results:
- Example:
   ```python
   state["documents"] = ["MS Dhoni is a former Indian cricketer and captain."]
   ```

---

### **Summary**
The workflow dynamically processes queries and routes them based on content:
1. Determines the route (`wiki_search` or `vectorstore`).
2. Fetches relevant documents.
3. Produces a structured final state with documents retrieved.

### **Comparison: `pprint` vs `print`**

#### **Data Structure**
```python
data = {
    "name": "Alice",
    "age": 30,
    "hobbies": ["reading", "cycling", "hiking"],
    "address": {
        "city": "New York",
        "zip_code": "10001",
        "coordinates": {"lat": 40.7128, "long": -74.0060}
    }
}
```

---

#### **Using `print`**
```plaintext
{'name': 'Alice', 'age': 30, 'hobbies': ['reading', 'cycling', 'hiking'], 'address': {'city': 'New York', 'zip_code': '10001', 'coordinates': {'lat': 40.7128, 'long': -74.006}}}
```

---

#### **Using `pprint`**
```plaintext
{
    'name': 'Alice',
    'age': 30,
    'hobbies': ['reading', 'cycling', 'hiking'],
    'address': {
        'city': 'New York',
        'zip_code': '10001',
        'coordinates': {'lat': 40.7128, 'long': -74.006}
    }
}
```